<img src="http://193.175.187.164/static/OEP_logo_2_no_text.svg" alt="OpenEnergy Platform" height="100" width="100"  align="left"/>
<img src="http://reiner-lemoine-institut.de//wp-content/uploads/2015/09/rlilogo.png" alt="RLI" height="100" width="100" align="right"/>

# OpenEnergy Platform
<br>
## Usage of OpenEnergyPlatform API-Dialect (oedialect)
Repository: https://github.com/openego/oedialect

In [ ]:
__copyright__ = "Reiner Lemoine Institut"
__license__   = "GNU Affero General Public License Version 3 (AGPL-3.0)"
__url__       = "https://github.com/openego/data_processing/blob/master/LICENSE"
__author__    = "henhuy, Ludee"

In [ ]:
import pandas as pd
import getpass
import sqlalchemy as sa
from sqlalchemy.orm import sessionmaker
import oedialect

In [ ]:
# Whitespaces are not a problem for setting up the url!
user = input('Enter OEP-username:')
token = getpass.getpass('Token:')

In [ ]:
# Create Engine:
OEP_URL = 'oep.iks.cs.ovgu.de'
OED_STRING = f'postgresql+oedialect://{user}:{token}@{OEP_URL}'

engine = sa.create_engine(OED_STRING)
metadata = sa.MetaData(bind=engine)

## Setup Table

In [ ]:
tname = 'example_dialect_table'
sname = 'sandbox'

table = sa.Table(
    tname,
    metadata,
    sa.Column('name', sa.VARCHAR(50)),
    sa.Column('age', sa.INTEGER),
    schema=sname
)

## Create Table

In [ ]:
conn = engine.connect()
print('Connection established')
if not engine.dialect.has_table(conn, tname, sname):
    table.create()
    print('Created table')

## Insert into Table

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()
try:
    insert_statement = table.insert().values(
        [
            dict(name='Peter', age=25),
            dict(name='Inge', age=42),
            dict(name='Horst', age=36)
        ]
    )
    session.execute(insert_statement)
    session.commit()
    print('Insert successful!')
except Exception as e:
    session.rollback()
    raise
finally:
    session.close()

## Select from Table

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()
print(session.query(table).all())
session.close()

In [ ]:
df = pd.DataFrame(session.query(table).all())
df